In [1]:
import re
import pandas as pd
import requests
import pdfplumber

In [2]:
arquivo = pdfplumber.open("arquivo/Extrato Banco.pdf")

In [3]:
with pdfplumber.open("arquivo/Extrato Agrupado Simplificado.pdf") as pdf:
    a = pdf.pages[0]
    a.chars[0]
    

In [4]:
b = a.extract_text().split("\n")

In [5]:
serie = pd.Series(b)

In [6]:
contratos = []
aluguel = []

In [7]:
for x in range (0, len(serie)):
    linha = serie.loc[x]
    primeira_palavra = linha.split(" ")[0]
    
    if primeira_palavra == "Emitido":
        data_emicao = linha.split(" ")[2]
        
    if primeira_palavra == "Contrato":
        contratos.append(linha[len("Contrato "):])
        
    if primeira_palavra == "Aluguel":
        aluguel.append(float((linha[len("Aluguel - 01/12/2020 até 31/12/2020 "):].replace(",", "."))))

In [8]:
aluguel

[368.3, 368.3, 429.68, 437.77]

<h1>Desafio 1</h1>

In [9]:
with pdfplumber.open("arquivo/Rua Ângelo Martins, 445 1-10.pdf") as pdf:
    RAM = pdf.pages[0]
    RAM.chars[0]

In [10]:
text_RAM = RAM.extract_text().split("\n")

In [11]:
text_RAM

['SEMEC - PREFEITURA MUNICIPAL DE MACEIO',
 'SECRETARIA MUNICIPAL DE ECONOMIA - SEMEC',
 'DOCUMENTO DE ARRECADAÇÃO MUNICIPAL - D.A.M MENSAL Text',
 'Perdeu o prazo do DAM? Emita automaticamente através do QRCODE',
 'Contribuinte D.A.M.',
 'MANOEL CORREIA TELLES 66114110 / 2021',
 'CPF/CNPJ Inscrição / C.M.C. Tributos Referência Vencimento Nosso Número',
 '002.643.184-04 000000000120433 I.P.T.U. 2021 1/10 31/03/2021 11000000075755253',
 'Endereço de Localização',
 'Logradouro: RUA - ÂNGELO MARTINS 00445 Cep: 57030592',
 'Número:',
 'Bairro: PONTA DA TERRA Complemento: Lote: 0530 Quadra: 0102',
 'Loteamento: Quadra Lot.: Lote Lot.:',
 'Imóvel: PREDIAL Identificação: 05010240530001',
 'Área Terreno: 119,00 Área Total Construída: 73,58 Área Construída Unidade: 73,58',
 'Valor Venal Terreno: 24.273,80 Valor Venal Construção: 51.307,41',
 'Valor Venal Imóvel: 75.581,21 Base de Cálculo: 45.348,73 Alíquota:  1,0000',
 'IPTU PREDIAL - PRINCIPAL 45,35',
 'COLETA DE LIXO 25,52 Valor R$ 70,87',
 '

In [12]:
tira_rep = 0
for x in range (0, len(text_RAM)):
    
    #nome do contribuinte
    if text_RAM[x] == 'Contribuinte D.A.M.':
        a = text_RAM[x+1]
    #CPF/CNPJ  //  Inscrição/CMC  //  Tributos  //  Referência  //  Vencimento
    if text_RAM[x][:8] == 'CPF/CNPJ':
        if tira_rep == 0:
            tira_rep = 1
            dados = text_RAM[x+1].split(" ")
            cpf_cnpj = dados[0]
            inscricao_CMC = dados[1]
            tributos = dados[2] + " " + dados[3]
            referencia = dados[4]
            vencimento = dados[5]
            print(vencimento)
    #Logradouro
    #Número
    #CEP
    #Bairro
    #Complemento
    #Área terreno
    #Área Total Construída
    #Área Construída Unidade
    #Valor Venal Terreno
    #Valor Venal Construção
    #Valor Venal Imóvel 
    #Base de Cálculo
    #Alíquota
    
    #total a pagar
    if text_RAM[x][0:len('Total a Pagar: ')] == 'Total a Pagar: ':
        total_a_pagar = float(text_RAM[x][len('Total a Pagar: '):].replace(',', '.'))

31/03/2021


In [13]:
total_a_pagar

70.87

In [14]:
codBarras = text_RAM[-1]

In [15]:
codBarras

'81620000000-7 70872485202-3 10331110000-4 00075755253-4'

In [16]:
def extrair(pdf):
    tira_rep = 0
    for x in range (0, len(pdf)):

        #nome do contribuinte
        if pdf[x] == 'Contribuinte D.A.M.':
            nome = text_RAM[x+1].upper()
            nome = re.sub('[^A-Z]', ' ', nome).strip(" ")
        #CPF/CNPJ  //  Inscrição/CMC  //  Tributos  //  Referência  //  Vencimento
        if pdf[x][:8] == 'CPF/CNPJ':
            if tira_rep == 0:
                tira_rep = 1
                dados = pdf[x+1].split(" ")
                cpf_cnpj = dados[0]
                inscricao_CMC = dados[1]
                tributos = dados[2] + " " + dados[3]
                referencia = dados[4]
                vencimento = dados[5]
        #Logradouro
        #Número
        #CEP
        #Bairro
        #Complemento
        #Área terreno
        #Área Total Construída
        #Área Construída Unidade
        #Valor Venal Terreno
        #Valor Venal Construção
        #Valor Venal Imóvel 
        #Base de Cálculo
        #Alíquota

        #total a pagar
        if pdf[x][0:len('Total a Pagar: ')] == 'Total a Pagar: ':
            total_a_pagar = float(pdf[x][len('Total a Pagar: '):].replace(',', '.'))
        codBarras = pdf[-1]
        
    dados_para_df = {'Nome':[nome],"cpf_cnpj":[cpf_cnpj],"inscricao_CMC":[inscricao_CMC],"tributos":tributos,"referencia":[referencia],"vencimento":[vencimento],"total_a_pagar":[total_a_pagar],'codBarras':[codBarras]}
    df = pd.DataFrame(dados_para_df).T
    return df

In [17]:
extrair(text_RAM)

,0
Nome,MANOEL CORREIA TELLES
cpf_cnpj,002.643.184-04
inscricao_CMC,000000000120433
tributos,I.P.T.U. 2021
referencia,1/10
vencimento,31/03/2021
total_a_pagar,70.87
codBarras,81620000000-7 70872485202-3 10331110000-4 0007...


In [21]:
def escrever(nome_arquivo):
    with pdfplumber.open("arquivo/"+nome_arquivo) as pdf:

        RAM_u = pdf.pages[-1]
        RAM_u.chars[0]
        text_RAM_u = RAM_u.extract_text().split("\n")
        
    return text_RAM_u

In [19]:
escrever("Extrato Agrupado Simplificado.pdf")

['Emitido em 23/01/2021 11:31:49',
 'Total  7.770,57 -1.096,01 6.674,56',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 'Zampieri Aluguéis',
 'Rua Durval Guimarães, 1738  - Ponta Verde',
 'Fone (82) 2121-6000 - Maceió / AL -',
 '3 de 3']